In [92]:
import pandas as pd 

training = pd.read_csv('./training.csv', usecols=["emotion", "image"," pixels"])
validation = pd.read_csv('./validation.csv', usecols=["emotion", "image"," pixels"])
test = pd.read_csv('./test.csv', usecols=["emotion", "image"," pixels"])


In [192]:
from PIL import Image
import numpy as np
from os import path
import torch
import os
import matplotlib.pyplot as plt # for debugging
from matplotlib import image
from matplotlib import pyplot

"""
Transforms the imgs in the training/testing/validation dataset and saves them as tensors

Arguments:
dataset: 'train', 'test', or 'validate'
num_imgs: number of imgs/pixels in the csv dataset

"""

def resize_and_flip(dataset, num_imgs):
    
    if dataset=='test':
        df = test
    elif dataset=='validate':
        df = validation
    elif dataset=='train':
        df = training
    else:
        print("ERR: dataset input is incorrect")
        return False
    
    for i in range(0, num_imgs):
        img_pixels = df.iloc[i][" pixels"]
        
        # if images are jpg 
        if type(img_pixels) is float:
            imgpath = './MuxspaceDataset/images/'
            img_pixels = image.imread(imgpath + str(df.iloc[i]["image"]))   # load image as pixel array
            img_data = np.resize(img_pixels, (48,48))
            img_tensor = torch.from_numpy(img_data)
            img_tensor_flipped = torch.from_numpy(np.fliplr(img_data).copy())
           
        # if images are pixels
        else:
            img_pixels = df.iloc[i][" pixels"]
            img_string = img_pixels.split(' ')
            img_data = np.asarray(img_string, dtype=np.uint8).reshape(48, 48)
            img_tensor = torch.from_numpy(img_data)
            img_tensor_flipped = torch.from_numpy(np.fliplr(img_data).copy())
        
        # save resized img as tensor
        master_path = './ProcessedData/' + str(dataset) + '/'
        folder_name = master_path + str(df.iloc[i]["emotion"])
        if not os.path.isdir(folder_name):
            os.mkdir(folder_name)
        torch.save(img_tensor, folder_name + '/' + str(i) + '.tensor')
        
        # save flipped and resized img as tensor
        torch.save(img_tensor_flipped, folder_name + '/' + str(num_imgs+i) + '.tensor')
        
    return True



In [193]:
resize_and_flip(dataset='train', num_imgs=3696)
resize_and_flip(dataset='validate', num_imgs=791)
resize_and_flip(dataset='test', num_imgs=798)

True

In [186]:
"""
Example of using dataloader on test images

"""

import torchvision

# Load Tensor Files (tensors) from folder

#load tensors
master_path = './ProcessedData/test/'
dataset = torchvision.datasets.DatasetFolder(master_path, loader=torch.load, extensions=('.tensor'))

# Prepare Dataloader
batch_size = 32
num_workers = 1
test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)

# Verification Step - obtain one batch of imgs
dataiter = iter(test_loader)
img, labels = dataiter.next()
print(img.shape)
print(labels.shape)

torch.Size([32, 48, 48])
torch.Size([32])
